In [1]:
import pandas as pd
import scrapy
from scrapy.item import Item, Field
import logging

from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [2]:
class EditalItem(Item):
    date = Field()
    tags = Field()
    title = Field()
    cover = Field()
    summary = Field()
    pdf = Field()

In [3]:
class EditaisSpider(scrapy.Spider):
    max_pages = 2
    name = 'secult'
    start_urls = ['http://editais.cultura.ce.gov.br/category/editais/page/%s' % page for page in range(1, max_pages + 1)]

    def parse(self, response):
        for edital in response.css('article'):
            url = edital.xpath('h2/a/@href').get()
            # get date from url
            date = url.replace('http://editais.cultura.ce.gov.br/', '').split('/')
            # change month to corresponding number
            date = f'{date[0]}-{date[1]}-{date[2]}'
            
            item = EditalItem()
            item['date'] = date
            item['tags'] = edital.xpath('p/a[@rel="category tag"]/text()').extract()
            yield scrapy.Request(url, callback=self.parse_edital, meta={'item':item})
#         next_page = response.css('div.pagination').xpath('div/a/@href').get()
#         if next_page is not None :
#             yield response.follow(next_page, self.parse)
    def parse_edital(self, response):
        item = response.meta['item']
        item['title'] = response.css('h1.entry-title::text').get()
        item['cover'] = response.css('article img::attr("src")').get()
        item['summary'] = response.css('div.et_pb_text_inner p ::text').get().strip()
        yield item
        
            

In [4]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(EditaisSpider)
process.start()

2021-12-12 14:29:10 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-12-12 14:29:10 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.9 (main, Nov 20 2021, 21:30:06) - [GCC 11.1.0], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.0, Platform Linux-5.15.7-arch1-1-x86_64-with-glibc2.33
2021-12-12 14:29:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-12-12 14:29:10 [scrapy.crawler] INFO: Overridden settings:
{}
2021-12-12 14:29:10 [scrapy.extensions.telnet] INFO: Telnet Password: c478408584e23289
2021-12-12 14:29:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-12-12 14:29:10 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermi

2021-12-12 14:29:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://editais.cultura.ce.gov.br/2021/02/03/xiii-edital-mecenas-do-ceara/> (referer: http://editais.cultura.ce.gov.br/category/editais/)
2021-12-12 14:29:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://editais.cultura.ce.gov.br/2021/11/19/chamada-publica-para-selecao-do-iv-seminario-de-formacao-avaliacao-e-planejamento-do-ciclo-carnavalesco-do-ceara-e-o-iv-seminario-de-avaliacao-e-planejamento-ceara-da-paixao-2022/> (referer: http://editais.cultura.ce.gov.br/category/editais/)
2021-12-12 14:29:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://editais.cultura.ce.gov.br/2019/08/27/eleicoes-cepc-2020-2022/> (referer: http://editais.cultura.ce.gov.br/category/editais/page/2/)
2021-12-12 14:29:15 [scrapy.core.scraper] DEBUG: Scraped from <200 http://editais.cultura.ce.gov.br/2019/01/08/chamada-publica-para-credenciamento-de-pareceristas-para-o-xiii-edital-carnaval-do-ceara-2019/>
{'cover': 'http://editais.cul